In [171]:
#project originally done in google collab hence mounted on drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#import necessary libraries
import sys
import cv2
import numpy as np
import random
from matplotlib import pyplot
#from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# define cnn model
def define_model(W,L):
  model = Sequential()
  model.add(Flatten(input_shape=(20,20,3)))
  model.add(Dense(20, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(W)))
  model.add(Dense(2, activation='softmax'))
  # compile model
  opt = SGD(learning_rate=L, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model
  

In [174]:
# obtaining the dataset
trainsize=1000
testsize=100

############insert path of the dataset folder
path=r'/content/drive/My Drive/dataset/'
img_face_train=[]
for i in range(trainsize//2):
    pat= path + 'dataset_face_train/im{}.jpg'.format(i+1)
    image=cv2.imread(pat)
    img_face_train.append(image)
img_face_train=np.array(img_face_train)    


img_face_test=[]
for i in range(testsize//2):
    pat= path + 'dataset_face_test/im{}.jpg'.format(i+1)
    image=cv2.imread(pat)
    img_face_test.append(image)
img_face_test=np.array(img_face_test)


img_nonface_train=[]
for i in range(trainsize//2):
    pat= path + 'dataset_nonface_train/im{}.jpg'.format(i+1)
    image=cv2.imread(pat)
    img_nonface_train.append(image)
img_nonface_train=np.array(img_nonface_train)


img_nonface_test=[]
for i in range(testsize//2):
    pat= path + 'dataset_nonface_test/im{}.jpg'.format(i+1)
    image=cv2.imread(pat)
    img_nonface_test.append(image)
img_nonface_test=np.array(img_nonface_test)

trainY=np.array([[0]]*(trainsize//2) + [[1]]*(trainsize//2))
testY=np.array([[0]]*(testsize//2) + [[1]]*(testsize//2))
trainY = to_categorical(trainY)
testY = to_categorical(testY)
trainX=np.append(img_face_train,img_nonface_train,axis=0)
testX=np.append(img_face_test,img_nonface_test,axis=0)
print(trainX.shape,type(trainX),testX.shape,type(testX))


(1000, 20, 20, 3) <class 'numpy.ndarray'> (100, 20, 20, 3) <class 'numpy.ndarray'>


In [0]:
# convert from integers to floats
train_norm = trainX.astype('float32')
test_norm = testX.astype('float32')

# normalize to range 0-1
trainX = train_norm / 255.0
testX = test_norm / 255.0

#Normalization
trainX-=np.mean(trainX,axis=0)
trainX/=np.std(trainX,axis=0)
testX-=np.mean(testX,axis=0)
testX/=np.std(testX,axis=0)
batch_size=64


In [176]:
# Common Training module with normalized data and data augmentation
Weight_Decay=1e-6 ;Learning_Rate=0.00001
model = define_model(Weight_Decay,Learning_Rate)
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,)
datagen.fit(trainX)
model.fit_generator(datagen.flow(trainX,trainY, batch_size=64),
                    steps_per_epoch=trainX.shape[0] // batch_size,epochs=10,
                    verbose=1,validation_data=(testX,testY))

Epoch 1/10
15/15 [==============================] - 0s 25ms/step - loss: 0.8272 - accuracy: 0.4904 - val_loss: 0.7540 - val_accuracy: 0.5000
Epoch 2/10
15/15 [==============================] - 0s 25ms/step - loss: 0.8061 - accuracy: 0.5021 - val_loss: 0.7327 - val_accuracy: 0.5200
Epoch 3/10
15/15 [==============================] - 0s 25ms/step - loss: 0.7805 - accuracy: 0.5208 - val_loss: 0.7115 - val_accuracy: 0.5500
Epoch 4/10
15/15 [==============================] - 0s 24ms/step - loss: 0.7362 - accuracy: 0.5641 - val_loss: 0.6936 - val_accuracy: 0.5600
Epoch 5/10
15/15 [==============================] - 0s 26ms/step - loss: 0.7314 - accuracy: 0.5641 - val_loss: 0.6765 - val_accuracy: 0.5800
Epoch 6/10
15/15 [==============================] - 0s 25ms/step - loss: 0.7267 - accuracy: 0.5577 - val_loss: 0.6617 - val_accuracy: 0.5900
Epoch 7/10
15/15 [==============================] - 0s 26ms/step - loss: 0.7080 - accuracy: 0.5813 - val_loss: 0.6484 - val_accuracy: 0.6000
Epoch 8/10
15

In [177]:
#train to overfit
Weight_Decay=1e-6 ;Learning_Rate=0.01
model = define_model(Weight_Decay,Learning_Rate)
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,)
datagen.fit(trainX)
model.fit_generator(datagen.flow(trainX,trainY, batch_size=64),
                    steps_per_epoch=trainX.shape[0] // batch_size,epochs=100,
                    verbose=1,validation_data=(testX,testY))

Epoch 1/100
15/15 [==============================] - 0s 24ms/step - loss: 0.5313 - accuracy: 0.7896 - val_loss: 0.1491 - val_accuracy: 0.9400
Epoch 2/100
15/15 [==============================] - 0s 24ms/step - loss: 0.2420 - accuracy: 0.9090 - val_loss: 0.0967 - val_accuracy: 0.9500
Epoch 3/100
15/15 [==============================] - 0s 25ms/step - loss: 0.2155 - accuracy: 0.9103 - val_loss: 0.0797 - val_accuracy: 0.9800
Epoch 4/100
15/15 [==============================] - 0s 26ms/step - loss: 0.1783 - accuracy: 0.9284 - val_loss: 0.0708 - val_accuracy: 0.9700
Epoch 5/100
15/15 [==============================] - 0s 27ms/step - loss: 0.1693 - accuracy: 0.9306 - val_loss: 0.0703 - val_accuracy: 0.9600
Epoch 6/100
15/15 [==============================] - 0s 25ms/step - loss: 0.1460 - accuracy: 0.9490 - val_loss: 0.0552 - val_accuracy: 0.9800
Epoch 7/100
15/15 [==============================] - 0s 25ms/step - loss: 0.1453 - accuracy: 0.9452 - val_loss: 0.0557 - val_accuracy: 0.9800
Epoch 

In [178]:
#4.	Sanity Check:Keep constant learning rate=0.001 and increase regularization
#(i) Regularization= 0 (no regularization)
Weight_Decay=0 ;Learning_Rate=0.001
model = define_model(Weight_Decay,Learning_Rate)
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,)
datagen.fit(trainX)
model.fit_generator(datagen.flow(trainX,trainY, batch_size=64),
                    steps_per_epoch=trainX.shape[0] // batch_size,epochs=10,
                    verbose=1,validation_data=(testX,testY))
#(ii) Regularization= 1000 (no regularization)
Weight_Decay=1000 ;Learning_Rate=0.001
model = define_model(Weight_Decay,Learning_Rate)
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,)
datagen.fit(trainX)
model.fit_generator(datagen.flow(trainX,trainY, batch_size=64),
                    steps_per_epoch=trainX.shape[0] // batch_size,epochs=10,
                    verbose=1,validation_data=(testX,testY))
#(iii) Regularization= 10000 (no regularization)
Weight_Decay=10000 ;Learning_Rate=0.001
model = define_model(Weight_Decay,Learning_Rate)
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,)
datagen.fit(trainX)
model.fit_generator(datagen.flow(trainX,trainY, batch_size=64),
                    steps_per_epoch=trainX.shape[0] // batch_size,epochs=10,
                    verbose=1,validation_data=(testX,testY))


Epoch 1/10
15/15 [==============================] - 0s 24ms/step - loss: 1.0008 - accuracy: 0.6368 - val_loss: 0.4378 - val_accuracy: 0.8300
Epoch 2/10
15/15 [==============================] - 0s 24ms/step - loss: 0.4529 - accuracy: 0.8237 - val_loss: 0.3230 - val_accuracy: 0.8500
Epoch 3/10
15/15 [==============================] - 0s 25ms/step - loss: 0.3172 - accuracy: 0.8600 - val_loss: 0.2546 - val_accuracy: 0.8800
Epoch 4/10
15/15 [==============================] - 0s 26ms/step - loss: 0.2754 - accuracy: 0.8846 - val_loss: 0.2255 - val_accuracy: 0.8800
Epoch 5/10
15/15 [==============================] - 0s 25ms/step - loss: 0.2304 - accuracy: 0.9167 - val_loss: 0.2062 - val_accuracy: 0.8900
Epoch 6/10
15/15 [==============================] - 0s 25ms/step - loss: 0.2392 - accuracy: 0.8964 - val_loss: 0.1876 - val_accuracy: 0.8900
Epoch 7/10
15/15 [==============================] - 0s 25ms/step - loss: 0.2205 - accuracy: 0.9115 - val_loss: 0.1708 - val_accuracy: 0.9200
Epoch 8/10
15

In [179]:
#5.	Sanity Check 2:For a constant Regularization=0.001 and change Learning 
#(i) Learning Rate=1e-6 
Weight_Decay=0.001 ;Learning_Rate=1e-6
model = define_model(Weight_Decay,Learning_Rate)
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,)
datagen.fit(trainX)
model.fit_generator(datagen.flow(trainX,trainY, batch_size=64),
                    steps_per_epoch=trainX.shape[0] // batch_size,epochs=10,
                    verbose=1,validation_data=(testX,testY))


#(i) Learning Rate=1e-3
Weight_Decay=0.001 ;Learning_Rate=1e-3
model = define_model(Weight_Decay,Learning_Rate)
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,)
datagen.fit(trainX)
model.fit_generator(datagen.flow(trainX,trainY, batch_size=64),
                    steps_per_epoch=trainX.shape[0] // batch_size,epochs=10,
                    verbose=1,validation_data=(testX,testY))

#(i) Learning Rate=1e1    
Weight_Decay=0.001 ;Learning_Rate=1e1
model = define_model(Weight_Decay,Learning_Rate)
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,)
datagen.fit(trainX)
model.fit_generator(datagen.flow(trainX,trainY, batch_size=64),
                    steps_per_epoch=trainX.shape[0] // batch_size,epochs=10,
                    verbose=1,validation_data=(testX,testY))


Epoch 1/10
15/15 [==============================] - 0s 24ms/step - loss: 0.9359 - accuracy: 0.4786 - val_loss: 0.8041 - val_accuracy: 0.4700
Epoch 2/10
15/15 [==============================] - 0s 24ms/step - loss: 0.9271 - accuracy: 0.4669 - val_loss: 0.8000 - val_accuracy: 0.4700
Epoch 3/10
15/15 [==============================] - 0s 26ms/step - loss: 0.9156 - accuracy: 0.4626 - val_loss: 0.7959 - val_accuracy: 0.4700
Epoch 4/10
15/15 [==============================] - 0s 25ms/step - loss: 0.9160 - accuracy: 0.4744 - val_loss: 0.7918 - val_accuracy: 0.4600
Epoch 5/10
15/15 [==============================] - 0s 25ms/step - loss: 0.9027 - accuracy: 0.4875 - val_loss: 0.7877 - val_accuracy: 0.4700
Epoch 6/10
15/15 [==============================] - 0s 26ms/step - loss: 0.9208 - accuracy: 0.4386 - val_loss: 0.7839 - val_accuracy: 0.4800
Epoch 7/10
15/15 [==============================] - 0s 25ms/step - loss: 0.8884 - accuracy: 0.4781 - val_loss: 0.7800 - val_accuracy: 0.4800
Epoch 8/10
15

In [180]:
#Hyperparameter Tuning
max_count=10
for count in range(max_count):
  REG=10**random.uniform(-6,0)
  LR=10**random.uniform(-6,-1)  
  model = define_model(REG,LR)
  print('Regularization=',REG,'Learning Rate=',LR)
  datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,)
  datagen.fit(trainX)
  model.fit_generator(datagen.flow(trainX,trainY, batch_size=64),
                      steps_per_epoch=trainX.shape[0] // batch_size,epochs=5,
                      verbose=1,validation_data=(testX,testY))

Regularization= 1.378918119766699e-06 Learning Rate= 0.006672752695761951
Epoch 1/5
15/15 [==============================] - 0s 24ms/step - loss: 0.4919 - accuracy: 0.7714 - val_loss: 0.2484 - val_accuracy: 0.8800
Epoch 2/5
15/15 [==============================] - 0s 24ms/step - loss: 0.2781 - accuracy: 0.8846 - val_loss: 0.1506 - val_accuracy: 0.9400
Epoch 3/5
15/15 [==============================] - 0s 25ms/step - loss: 0.2116 - accuracy: 0.9167 - val_loss: 0.1377 - val_accuracy: 0.9400
Epoch 4/5
15/15 [==============================] - 0s 25ms/step - loss: 0.1861 - accuracy: 0.9323 - val_loss: 0.1166 - val_accuracy: 0.9600
Epoch 5/5
15/15 [==============================] - 0s 25ms/step - loss: 0.1642 - accuracy: 0.9397 - val_loss: 0.1054 - val_accuracy: 0.9600
Regularization= 0.004672769145164078 Learning Rate= 0.06954363238679744
Epoch 1/5
15/15 [==============================] - 0s 24ms/step - loss: 1.2090 - accuracy: 0.7667 - val_loss: 1.0907 - val_accuracy: 0.9200
Epoch 2/5
15/1

In [181]:
# Train for most suitable value Regularization= 0.02597090269567742 Learning Rate= 0.05210002876562015
Weight_Decay=0.02597090269567742 ;Learning_Rate=0.05210002876562015
model = define_model(Weight_Decay,Learning_Rate)
datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True,)
datagen.fit(trainX)
model.fit_generator(datagen.flow(trainX,trainY, batch_size=64),
                    steps_per_epoch=trainX.shape[0] // batch_size,epochs=10,
                    verbose=1,validation_data=(testX,testY))


Epoch 1/10
15/15 [==============================] - 0s 24ms/step - loss: 1.4844 - accuracy: 0.8034 - val_loss: 1.0911 - val_accuracy: 0.8500
Epoch 2/10
15/15 [==============================] - 0s 23ms/step - loss: 0.9318 - accuracy: 0.8825 - val_loss: 0.5334 - val_accuracy: 0.9700
Epoch 3/10
15/15 [==============================] - 0s 26ms/step - loss: 0.5268 - accuracy: 0.9220 - val_loss: 0.3055 - val_accuracy: 0.9600
Epoch 4/10
15/15 [==============================] - 0s 25ms/step - loss: 0.3756 - accuracy: 0.9188 - val_loss: 0.2627 - val_accuracy: 0.9400
Epoch 5/10
15/15 [==============================] - 0s 25ms/step - loss: 0.3242 - accuracy: 0.9145 - val_loss: 0.2166 - val_accuracy: 0.9600
Epoch 6/10
15/15 [==============================] - 0s 26ms/step - loss: 0.2703 - accuracy: 0.9354 - val_loss: 0.2331 - val_accuracy: 0.9300
Epoch 7/10
15/15 [==============================] - 0s 26ms/step - loss: 0.3186 - accuracy: 0.9135 - val_loss: 0.2446 - val_accuracy: 0.9500
Epoch 8/10
15

In [182]:
# evaluate model
_, acc = model.evaluate(testX, testY, verbose=0)
print('> %.3f' % (acc * 100.0))
# learning curves
summarize_diagnostics(history)

> 97.000
